In [48]:
using BenchmarkTools

# Create example array
S = (rand(4,4,1000) + 1im .* rand(4,4,1000)) ./ sqrt(2);

# Manual inversion
function f(A)
       B = similar(A)
       for i in 1:size(A,3)
           B[:,:,i] = inv(@view A[:,:,i])
       end
       return B
end

@btime f($S)

  982.500 μs (4002 allocations: 4.73 MiB)


4×4×1000 Array{ComplexF64, 3}:
[:, :, 1] =
  2.04396+0.516337im  -3.53941+0.586511im  …  -0.726118-0.693697im
 -2.53882+2.5512im     4.21057-3.83054im        1.60783-0.541738im
 -1.34086-2.58766im    2.14089+2.63748im      -0.817487+1.26573im
  1.89067-2.24802im   -2.65782+2.62374im        0.15483+0.0033661im

[:, :, 2] =
 -0.220214+0.656791im  -0.790315+1.78539im   …    1.38218-1.5672im
  -1.02722-2.50538im   -0.259431-0.802446im      0.692298+1.87792im
   1.62027+1.24407im    0.441131-0.524186im      -1.98119-0.900273im
  0.477622+0.592742im   0.880625-1.01122im      -0.386291+0.429986im

[:, :, 3] =
 0.430582-1.74381im    -0.805571+1.27898im   …  -0.559463+0.282339im
 -1.46814-1.34311im     0.110164-0.649686im       1.55936+1.31881im
  1.05403+0.823496im  -0.0738271-0.774333im      -1.84597+0.0992289im
  1.25176+1.65204im      1.19085-0.700634im      0.195617-2.07334im

...

[:, :, 998] =
 -1.58752-2.57738im    0.933775+0.064455im  …   2.37117+2.92093im
    1.025-1.02955im   0.04211

In [47]:
function g(A)
    B = similar(A)
    a = similar(A, size(A)[1:2])
    u = one(a)
    for i in 1:size(A,3)
        a = @view A[:,:,i]
        b = @view B[:,:,i]
        b .= inv(a)
    end
    return B
end

@btime g($S)

  969.400 μs (4004 allocations: 4.73 MiB)


4×4×1000 Array{ComplexF64, 3}:
[:, :, 1] =
  0.127666-0.348605im   0.894626+0.0254583im  …    1.51883+0.576025im
 -0.825056+0.101675im  -0.339885-0.796863im         -1.072+0.0314285im
 -0.320084-1.39379im    -0.22788+1.34349im        0.420813-0.971523im
  0.964716+1.41175im    0.137543-0.379229im      -0.716802-0.169122im

[:, :, 2] =
 -0.316258-0.428817im  -0.871715-0.391721im  …   0.820968+0.332952im
 -0.748023+0.196345im   0.478482+0.500139im     0.0977272-1.20135im
 -0.967822-0.758657im   0.171586+0.547532im       1.01983+0.651859im
   1.73443+0.457735im   0.565718-0.767152im      -1.17427-0.220548im

[:, :, 3] =
 0.913889+1.66466im     -1.59164+0.0333835im  …  -0.146432+0.733381im
 -1.02689+0.332067im   -0.521639-0.914095im       0.375789+1.35817im
  1.32173-1.89765im      0.61705+1.30942im          1.4775-2.22977im
 -0.57877-0.0515324im    1.58378-0.806761im       -0.83986+0.060374im

...

[:, :, 998] =
  -1.1278-0.0112787im   -1.28862+0.542508im  …   0.901639-0.922481im
 -0.5593

In [2]:
@btime mapslices(inv, $S; dims=(1,2))

  1.216 ms (11504 allocations: 4.91 MiB)


4×4×1000 Array{ComplexF64, 3}:
[:, :, 1] =
 -0.472938-1.02225im    0.239356-1.4069im    …   -2.40668+0.0198024im
 0.0998238+1.08809im    -1.07009+3.35153im       0.969672-0.773027im
  -1.39954-1.30705im    0.280531-0.411809im     -0.523916+0.206692im
   1.90878-0.0184063im  0.290608-2.11538im        1.90141+0.377283im

[:, :, 2] =
 0.780821+0.400241im   -1.51131-0.938182im  …   2.17192+0.311844im
 0.668039+0.205166im   0.611185+1.17768im      -2.56644+0.983934im
 0.206696+0.397545im    1.06807-0.568323im     -1.39854-1.14737im
  -1.1903-0.864455im  -0.637007-0.273049im      2.79862-0.888559im

[:, :, 3] =
 -3.78136-1.07419im     2.31221-1.3541im    …   3.65273+2.12815im
  3.90895-1.97111im    -2.10226+3.86691im      -1.95574-1.05314im
 0.658292+0.765656im   0.729538-3.08042im      -1.96464+1.62462im
 -7.23525+0.0401632im   5.54892-0.343697im      2.71885-1.25873im

...

[:, :, 998] =
  4.71058+1.60934im    1.23053+2.56591im  …  -4.05088-0.252429im
  2.50383+9.36604im   -1.21521-1.50842

In [3]:
@code_warntype mapslices(inv, S; dims=(1,2))

Variables
  #unused#::Core.Const(Base.var"#mapslices##kw"())
  @_2::NamedTuple{(:dims,), Tuple{Tuple{Int64, Int64}}}
  @_3::Core.Const(mapslices)
  f::Core.Const(inv)
  A::Array{ComplexF64, 3}
  dims::Tuple{Int64, Int64}
  @_7::Tuple{Int64, Int64}

Body::Any
1 ─ %1  = Base.haskey(@_2, :dims)::Core.Const(true)
│         %1
│         (@_7 = Base.getindex(@_2, :dims))
└──       goto #3
2 ─       Core.Const(:(Core.UndefKeywordError(:dims)))
└──       Core.Const(:(@_7 = Core.throw(%5)))
3 ┄ %7  = @_7::Tuple{Int64, Int64}
│         (dims = %7)
│   %9  = (:dims,)::Core.Const((:dims,))
│   %10 = Core.apply_type(Core.NamedTuple, %9)::Core.Const(NamedTuple{(:dims,), T} where T<:Tuple)
│   %11 = Base.structdiff(@_2, %10)::Core.Const(NamedTuple())
│   %12 = Base.pairs(%11)::Core.Const(Base.Iterators.Pairs{Union{}, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())
│   %13 = Base.isempty(%12)::Core.Const(true)
│         %13
└──       goto #5
4 ─       Core.Const(:(Base.kwerr(@_2, @_3, f, A)))
5 ┄ %17 = 

In [52]:
@code_warntype mapslices(identity, S; dims=(1,2))

Variables
  #unused#::Core.Const(Base.var"#mapslices##kw"())
  @_2::NamedTuple{(:dims,), Tuple{Tuple{Int64, Int64}}}
  @_3::Core.Const(mapslices)
  f::Core.Const(identity)
  A::Array{ComplexF64, 3}
  dims::Tuple{Int64, Int64}
  @_7::Tuple{Int64, Int64}

Body::Any
1 ─ %1  = Base.haskey(@_2, :dims)::Core.Const(true)
│         %1
│         (@_7 = Base.getindex(@_2, :dims))
└──       goto #3
2 ─       Core.Const(:(Core.UndefKeywordError(:dims)))
└──       Core.Const(:(@_7 = Core.throw(%5)))
3 ┄ %7  = @_7::Tuple{Int64, Int64}
│         (dims = %7)
│   %9  = (:dims,)::Core.Const((:dims,))
│   %10 = Core.apply_type(Core.NamedTuple, %9)::Core.Const(NamedTuple{(:dims,), T} where T<:Tuple)
│   %11 = Base.structdiff(@_2, %10)::Core.Const(NamedTuple())
│   %12 = Base.pairs(%11)::Core.Const(Base.Iterators.Pairs{Union{}, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())
│   %13 = Base.isempty(%12)::Core.Const(true)
│         %13
└──       goto #5
4 ─       Core.Const(:(Base.kwerr(@_2, @_3, f, A)))
5 ┄ %

In [55]:
@code_warntype Base.:(var"#mapslices#134")((1,2), mapslices, inv, S)

Variables
  #mapslices#134::Core.Const(Base.var"#mapslices#134")
  dims@_2::Tuple{Int64, Int64}
  @_3::Core.Const(mapslices)
  f::Core.Const(inv)
  A::Array{ComplexF64, 3}
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  #136::Base.var"#136#138"
  #135::Union{}
  @_9::Union{Nothing, Tuple{Int64, Int64}}
  indices::Base.Iterators.Drop{_A} where _A
  nidx::Int64
  ridx::Vector{Any}
  R::Any
  nextra::Any
  r1::Any
  tmp::Any
  Rsize::Vector{Base.OneTo{Int64}}
  safe_for_reuse::Bool
  Aslice::Any
  itershape::Tuple{Vararg{Base.OneTo{Int64}, N} where N}
  idx::Vector{Any}
  otherdims::Vector{Int64}
  alldims::Vector{Int64}
  ndimsA::Int64
  dimsA::Vector{Base.OneTo{Int64}}
  @_26::Union{Nothing, Tuple{Base.OneTo{Int64}, Int64}}
  @_27::Int64
  @_28::Vector{Any}
  ind::Base.OneTo{Int64}
  d@_30::Int64
  d@_31::Int64
  dims@_32::Union{Tuple{Int64, Int64}, Vector{Int64}}
  @_33::Bool
  @_34::Bool
  @_35::Bool
  @_36::Any

Body::Any
1 ──        nothing
│           (dims@_32 = dims@_2)
│           